In [1]:
f = open("philosophy_data.csv", "r").read()

In [2]:
lines = f.split("\n")

In [3]:
import re
from collections import defaultdict
from tqdm import tqdm

PATTERN = r',(?=(?:[^"]*"[^"]*")*[^"]*$)'

attr_handler: dict[str, callable] = {
    "title": str,
    "author": str,
    "school": str,
    "sentence_spacy": lambda x: str(x[1:-1]),
    "sentence_str":  lambda x: str(x[1:-1]),
    "original_publication_date": int,
    "corpus_edition_date": int,
    "sentence_length": int,
    "sentence_lowered": lambda x: str(x[1:-1]),
    "tokenized_txt": lambda x: list(x[2:-2].replace("\'", "").split(", ")),
    "lemmatized_str": lambda x: str(x[1:-1])
}

def split_lines(lines):
    headers = lines[0].split(",")
    jsonl = []
    for line in tqdm(lines[1:]):
        split_line = re.split(PATTERN, line)
        if len(headers) != len(split_line):
            continue
        jsonl.append({header: attr_handler[header](split_line[i]) for i, header in enumerate(headers)})
    return jsonl

In [9]:
di_lines = split_lines(lines)

100%|██████████| 396428/396428 [00:49<00:00, 7975.81it/s] 


In [14]:
import jsonlines
with jsonlines.open("./data/index.jsonl", mode="w") as writer:
    writer.write(di_lines)

In [5]:
import jsonlines
examples = None
with jsonlines.open("./data/index.jsonl", mode="r") as reader:
    examples = [li for li in reader][0]

In [7]:
from collections import defaultdict

def partition(examples, key):
    part = defaultdict(list)
    for example in examples:
        if key not in example:
            continue
        part[example[key]].append(example)
    return part

In [8]:
schools = partition(examples, "school")

In [18]:
def group(part, k):
    groups = []
    for g in part:
        print(g)
        for i in range(len(part[g]) - k):
            groups.append(part[g][i:i+k])
    return groups

In [19]:
schools_len_k = group(schools, 5)

plato
aristotle
empiricism
rationalism
analytic
continental
phenomenology
german_idealism
communism
capitalism
stoicism
nietzsche
feminism
scholasticism
Kierkegaard
Hobbes
Existentialism
Daoism


In [21]:
".".join([s["sentence_lowered"] for s in schools_len_k[1]])

'urely you are not prosecuting anyone before the king archon as i am.the athenians do not call this a prosecution but an indictment, euthyphro..hat is this you say.someone must have indicted you, for you are not going to tell me that you have indicted someone else..ut someone else has indicted you'

In [ ]:
Who wrote the following?

By John Locke

Certainly, you aren't presenting a case against someone before the King John Locke in the same manner that I am. It's important to note, John Locke, that the John Locke don't refer to this process as a prosecution but rather as an indictment. What do you mean by that? It must be the case that someone has brought charges against you, for it's hard to believe that you would be the one to accuse another. Indeed, it is the case that another has brought an indictment against you.

6. Plato
1. John Locke
2. Socrates
3. Aristotle
6. Plato
4. Sophocles
5. Thucydides